In [ ]:
import numpy as np
import pandas as pd
import scipy.stats
from tqdm import tqdm
import matplotlib.pyplot as pl

from fair21.energy_balance_model import EnergyBalanceModel

In [ ]:
temp_in = np.load('../data/ar6_ensemble_batches/temperature_1850-2030.npy')
ohc_in = np.load('../data/ar6_ensemble_batches/ohc_2018_minus_1971.npy')
co2_in = np.load('../data/ar6_ensemble_batches/co2_2014.npy')
ari_in = np.load('../data/ar6_ensemble_batches/fari_2005-2014_mean.npy')
aci_in = np.load('../data/ar6_ensemble_batches/faci_2005-2014_mean.npy')
o3_in = np.load('../data/ar6_ensemble_batches/fo3_2019.npy')
ecs_in = np.load('../data/ar6_ensemble_batches/ecs.npy')
tcr_in = np.load('../data/ar6_ensemble_batches/tcr.npy')

In [ ]:
df_cc=pd.read_csv('../data/parameter_sets/carbon_cycle.csv')
df_cr=pd.read_csv('../data/parameter_sets/climate_response.csv')
df_aci=pd.read_csv('../data/parameter_sets/erfaci.csv')
df_ari=pd.read_csv('../data/parameter_sets/erfari.csv')
df_ozone=pd.read_csv('../data/parameter_sets/ozone.csv')
df_scaling=pd.read_csv('../data/parameter_sets/forcing_scaling.csv')

In [ ]:
def rmse(obs, mod):
    return np.sqrt(np.sum((obs-mod)**2)/len(obs))

In [ ]:
temp_19952014 = temp_in[145:165,:].mean(axis=0)-temp_in[:51,:].mean(axis=0)

In [ ]:
samples = len(temp_19952014)

In [ ]:
df_gmst = pd.read_csv('../data/forcing/AR6_GMST.csv')
gmst = df_gmst['gmst'].values

In [ ]:
temp_rmse = np.ones((samples)) * np.nan
for i in range(samples):
    temp_rmse[i] = rmse(gmst[:171], temp_in[:171,i]-temp_in[:51, i].mean())

In [ ]:
NINETY_TO_ONESIGMA=scipy.stats.norm.ppf(0.95)
ecs_dist = scipy.stats.skewnorm(8.82185594, loc=1.95059779, scale=1.55584604)
tcr_dist = scipy.stats.norm(loc=1.2, scale=0.6/NINETY_TO_ONESIGMA)
temp_rmse_dist = scipy.stats.

In [ ]:
likelihood = np.zeros(samples)
for i in tqdm(range(samples)):
    likelihood[i] = ecs_dist.pdf(ecs_in[i]) * tcr_dist.pdf(tcr_in[i])

In [ ]:
pl.hist(likelihood)

In [ ]:
erf_2co2 = 3.934168323890023
calibrated_f4co2_mean = df_cr['F_4xCO2'].mean()

ecs = np.ones(samples) * np.nan
tcr = np.ones(samples) * np.nan
for iconf in tqdm(range(samples)):
    ebm = EnergyBalanceModel(
        ocean_heat_capacity = np.array([df_cr.loc[iconf,'c1'], df_cr.loc[iconf,'c2'], df_cr.loc[iconf, 'c3']]),
        ocean_heat_transfer = np.array([df_cr.loc[iconf,'kappa1'], df_cr.loc[iconf,'kappa2'], df_cr.loc[iconf,'kappa3']]),
        deep_ocean_efficacy = df_cr.loc[iconf,'epsilon'],
        forcing_4co2 = 2*erf_2co2 * (1+ 0.561*(calibrated_f4co2_mean - df_cr.loc[iconf,'F_4xCO2'])/calibrated_f4co2_mean) #2*erf_2co2*df_scale.loc[i,'CO2'],
    )
    ebm.emergent_parameters()
    ecs[iconf] = ebm.ecs
    tcr[iconf] = ebm.tcr

In [ ]:
pl.hist(ecs, bins=np.arange(0,15,0.1), density=True)
kde_ecs = scipy.stats.gaussian_kde(ecs)
pl.plot(np.arange(0,15,0.1), kde_ecs(np.arange(0,15,0.1)))

In [ ]:
np.histogram(ecs, bins=np.arange(0,15,0.1), density=True)[0]

In [ ]:
ecs[0]

In [ ]:
np.histogram(ecs, bins=np.arange(0,15,0.1), density=True)[0][np.floor(ecs[0:10]*10).astype(int)]

In [ ]:
kde_tcr = scipy.stats.gaussian_kde(tcr)

In [ ]:
prior = (
    np.histogram(ecs, bins=np.arange(0,25,0.1), density=True)[0][np.floor(ecs*10).astype(int)] *
    np.histogram(tcr, bins=np.arange(0,25,0.1), density=True)[0][np.floor(tcr*10).astype(int)]
)

In [ ]:
pl.hist(prior)

In [ ]:
posterior = prior * likelihood

In [ ]:
pl.hist(posterior)